In [ ]:
'''
create sql dataset from hdf5 files directly
   
@author - Farzan Memarian
 this is based on Andrew's id_title_artist.py and Alen's create_song_sql.py
'''
import hdf5_getters
from tqdm import tqdm
import pymysql.cursors
import pandas as pd
import numpy as np
import glob
import csv
import pdb

# establish connection to sql server
connection = pymysql.connect(host='localhost',\
   user='root',db='songs')
cursor = connection.cursor()

# create sql table (only need to do this once)
# sql = '''CREATE TABLE songs (

# songID VARCHAR(50) PRIMARY KEY, 
# artist VARCHAR(400) DEFAULT NULL,
# title VARCHAR(400) DEFAULT NULL,
# danceability REAL,
# duration REAL,
# energy REAL,
# loudness REAL,
# musicalKey INT,
# mode INT,
# tempo REAL,

# INDEX artist (artist)
# );'''
# cursor.execute(sql)
# connection.commit()

glob_path = '/home/frank/1CSEM/1UTCoursesTaken/dataMiningEE380L/termProject/dataset/MillionSongSubset/data/*/*/*/*'
filepaths = glob.glob(glob_path)
for filepath in tqdm(filepaths):
    h5 = hdf5_getters.open_h5_file_read(filepath)
    n = hdf5_getters.get_num_songs(h5)
    # print n
    for row in range(n):
        artist = hdf5_getters.get_artist_name(h5,songidx=row).decode('UTF-8')
        song_id = hdf5_getters.get_song_id(h5,songidx=row).decode('UTF-8')
        title= hdf5_getters.get_title(h5,songidx=row).decode('UTF-8')
        danceability = hdf5_getters.get_danceability(h5,songidx=row)
        duration = hdf5_getters.get_duration(h5,songidx=row)
        energy = hdf5_getters.get_energy(h5,songidx=row)
        loudness = hdf5_getters.get_loudness(h5,songidx=row)
        musicalKey = hdf5_getters.get_key(h5,songidx=row)
        mode = hdf5_getters.get_mode(h5,songidx=row)
        tempo = hdf5_getters.get_tempo(h5,songidx=row)
        artist_mbtags = hdf5_getters.get_artist_mbtags(h5,songidx=row)
        artist_mbtags_count = hdf5_getters.get_artist_mbtags_count(h5,songidx=row)
        tempo = hdf5_getters.get_tempo(h5,songidx=row)
        print(tempo)
        print(artist_mbtags)
        print(artist_mbtags_count)
        # info = [song_id, title, artist]
        # pdb.set_trace()
        # populate sql table with data
        query = "INSERT INTO songs (songID, artist, title, danceability, duration, energy, loudness, musicalKey,\
            mode, tempo) VALUES ('%s', '%s','%s','%s','%s','%s','%s','%s','%s','%s')"\
            % (song_id, artist.replace("'",""), title.replace("'",""), danceability, duration, energy,\
              loudness, musicalKey, mode, tempo)
        cursor.execute(query)
        connection.commit()
    h5.close()

# close sql connection
cursor.close()
connection.close()

  0%|          | 6/10000 [00:00<03:09, 52.63it/s]

84.96
[]
[]
150.04
[b'post-punk' b'uk' b'british' b'new wave' b'english' b'80s' b'goth'
 b'rock and indie' b'70s' b'manchester' b'britannique']
[4 3 3 2 2 2 1 1 1 1 1]
91.964
[]
[]
131.552
[]
[]
192.07
[]
[]
91.005
[b'world']
[1]
100.881
[]
[]
87.507
[]
[]
68.537
[]
[]
122.488
[b'alternative rock' b'rock' b'pop' b'indie rock' b'american'
 b'pop and chart']
[1 1 1 1 1 1]
94.257
[]
[]


  0%|          | 17/10000 [00:00<03:12, 51.74it/s]

110.0
[b'deutschland']
[1]
115.434
[]
[]
87.326
[b'hip hop rnb and dance hall' b'us' b'hiphop']
[1 1 1]
148.071
[b'uk' b'punk' b'england' b'bournemouth' b'anarcho']
[1 1 1 1 1]
167.941
[]
[]
90.811
[]
[]
61.634
[b'conductor']
[1]
121.948
[b'pop' b'pop and chart' b'am\xc3\xa9ricain' b'electronic' b'usa'
 b'american' b'singer' b'tell me' b'chanteur']
[3 1 1 1 1 1 1 1 1]
85.865
[b'german' b'hip hop']
[2 1]
200.289
[]
[]
144.424
[b'uk' b'english' b'british' b'punk' b'england' b'mod'
 b'classic pop and rock']
[2 1 1 1 1 1 1]


  0%|          | 29/10000 [00:00<03:12, 51.89it/s]

...

 12%|█▏        | 1184/10000 [00:23<02:56, 49.96it/s]

91.087
[b'operatic pop' b'gospel' b'classical' b'easy listening' b'american'
 b'vocal' b'pop' b'easy listening soundtracks and musicals']
[1 1 1 1 1 1 1 1]
84.606
[]
[]
35.351
[]
[]
113.41
[]
[]
64.996
[]
[]
88.011
[b'rock and indie']
[1]
124.915
[b'alternative rock' b'funk rock' b'funk' b'rock' b'crossover' b'10s'
 b'usa' b'00s' b'90s' b'80s' b'metal' b'rap' b'funk metal' b'american'
 b'pop and chart']
[3 3 2 2 1 1 1 1 1 1 1 1 1 1 1]
109.522
[b'hip hop rnb and dance hall' b'rnb' b'pop']
[1 1 1]
95.075
[]
[]
116.338
[b'finnish' b'humppa']
[2 1]
72.357
[]
[]


 12%|█▏        | 1196/10000 [00:23<02:51, 51.31it/s]

89.912
[]
[]
106.936
[]
[]
125.787
[]
[]
178.825
[]
[]
181.002
[b'betulio medina']
[1]
155.979
[b'greek']
[1]
80.945
[]
[]
177.963
[]
[]
98.988
[b'country' b'rock roll' b'rock' b'honky tonk' b'nashville sound'
 b'american' b'pop' b'rock and indie']
[1 1 1 1 1 1 1 1]
109.994
[]
[]
94.709
[]
[]


 12%|█▏        | 1202/10000 [00:23<02:50, 51.65it/s]